In [ ]:
"""
The purpose of this project is to evaluate how well macroeconomic factors at the time that a loan is originated predict
the loan's default likelihood. The below reads in a dataset of Lending Club loan data, including loan default statistics
and origination dates. It additionally reads in a dataset containing monthly data on the number of bankruptcies filed
nationally by month, stock market data, and unemployment data. Finally, the data sources are all combined for future analysis.
"""

In [1]:
# Read in the relevant libraries. In addition to Pandas, a library designed to connect to the FRED
# (Federal Reserve Economic Data) API is used.

import pandas as pd
import fredapi

In [ ]:
# Read the data from the Lending Club accepted loan dataset into a Pandas dataframe. The file
# was sourced from the following:
# https://www.kaggle.com/wordsforthewise/lending-club

local_directory = r'C:\Users\Mark\Desktop\springboard_projects\data'
lending_club_data = pd.read_csv(local_directory + r'\lending_club_accepted_loans.csv')

# The issue date needs to be converted from "mmm-yyyy" to the first of the month, to conform to the other data sources
lending_club_data['orig_month'] = pd.to_datetime('01-' + lending_club_data['issue_d'])


In [3]:
# Read the data from the monthly bankruptcy dataset into a Pandas dataframe. The file was
# sourced from the following:
# https://www.abi.org/newsroom/epiq-stats/april-2019-bankruptcy-statistics-state-and-district
local_directory = r'C:\Users\Mark\Desktop\springboard_projects\data'
bankruptcy_data = pd.read_excel(f'{local_directory}\\monthly_bk_data.xlsx', sheet_name='Filings by Jurisdiction',
                               header=None, skiprows=1, index_col=0)

# The original file has merged cells across the top representing years, and a "TOTAL" row across the bottom.
# The years need to extend throughout the merged cells, the years/months merged into a single index, all but the TOTAL
# rows removed, and the data rotated.

# Subset to only the year, month, and "TOTAL" rows. First, change the first two row names to "year" and "month"
new_index = bankruptcy_data.index.tolist()
new_index[0] = 'year'
new_index[1] = 'month'
bankruptcy_data.index = new_index

# Then, subset the df to only the year, month, and TOTAL

bk_data_subset = bankruptcy_data.loc[['year', 'month', 'TOTAL']]

# The 'year' row was the row of merged cells. The row must be forward-filled to replace NaN values with
# the correct years.

bk_data_subset.loc['year'] = bk_data_subset.loc['year'].fillna(method='ffill')

# Transpose the data
bk_data_subset = bk_data_subset.T

# Combine the year and month into a single column, which will be used as the date column
bk_data_subset['bk_month'] = bk_data_subset['month'] + ' 1 ' + bk_data_subset['year'].map(str)

# Convert the bk_month column to datetime.
bk_data_subset['bk_month'] = pd.to_datetime(bk_data_subset['bk_month'], format='%B %d %Y', errors='coerce')

# There are some rows that do not contain month-year-TOTAL data. Anything that is not a datetime in bk_month
# is one of these rows and is removed.
bk_data_subset = bk_data_subset.loc[~bk_data_subset['bk_month'].isnull()]

# The rows with zero total bankruptcies represent months that have not yet passed. They can be removed.
bk_data_subset = bk_data_subset.loc[bk_data_subset['TOTAL'] != 0]

# The dataframe can now be indexed using the bk_month column, have the 'year' and 'month' columns removed,
# and be sorted by the new index.
bk_data_subset = bk_data_subset[['TOTAL', 'bk_month']]
bk_data_subset.columns = ['monthly_bks', 'month']
bk_data_subset = bk_data_subset.set_index('month').sort_index()


In [6]:
bk_data_subset.head()
bk_data_subset.to_pickle(r'C:\Users\Mark\Desktop\springboard_projects\data\monthly_bk_data.pkl')

In [7]:
# The remainder of the data is sourced from the FRED API below. The unemployment data is formatted ideally for 

# Set up the API. The API key is removed from the published version.
API_KEY = ''
fred = fredapi.Fred(api_key=API_KEY)

# Pull the S&P 500 data
sp_data = fred.get_series('SP500').to_frame().reset_index()
sp_data.columns = ['date', 'value']

# There is no S&P data on some weekend/holiday days. To accommodate this, the data is forward filled.

# The data needs to be converted to a format that is usable at the monthly level. It will be summarized to
# only the close price on the first and last day of the month.
modified_sp = pd.DataFrame(index=sp_data['date'])
sp_data['bom'] = sp_data['date'].values.astype('<M8[M]')
sp_data['eom'] = pd.Index(sp_data['bom']).to_period('M').to_timestamp('M')

bom_values = sp_data[sp_data['bom'] == sp_data['date']][['date', 'value']]
eom_values = sp_data[sp_data['eom'] == sp_data['date']][['date', 'value']]
eom_values['date'] = eom_values['date'].values.astype('<M8[M]')

bom_values.columns = ['date', 'bom_value']
eom_values.columns = ['date', 'eom_value']

modified_sp = modified_sp.merge(right=bom_values, how='left', left_index=True, right_on='date')
modified_sp = modified_sp.merge(right=eom_values, how='left', left_on='date', right_on='date')

modified_sp = modified_sp.loc[~modified_sp['bom_value'].isnull()]

# There is no S&P data on some weekend/holiday days. To accommodate this, the data is forward filled.
modified_sp['bom_value'] = modified_sp['bom_value'].fillna(method='ffill')
modified_sp['eom_value'] = modified_sp['eom_value'].fillna(method='ffill')

# The average monthly stock price may be a better indicator for stock market performance than BOM or EOM
# values. Average monthly stock prices are pulled in as well.
bom_mean = sp_data[['bom', 'value']].groupby('bom').mean()
modified_sp = modified_sp.merge(bom_mean, left_on='date', right_index=True)
modified_sp = modified_sp.rename(columns={'value': 'month_avg'})

# Pull the unemployment data
unemployment_data = fred.get_series('UNRATE').to_frame()
unemployment_data.columns = ['unemployment_rate']


ValueError: Bad Request.  The value for variable api_key is not a 32 character alpha-numeric lower-case string.  Read https://research.stlouisfed.org/docs/api/api_key.html for more information.

In [ ]:
# For the purposes of this analysis, daily stock market volatility introduces noise which may
# obscure the true relationships between stock trends and default likelihood. In addition,
# the bankruptcy, origination, and unemployment data are only available at the monthly level.
# For these reasons, all data is sampled at the monthly level before being combined.

# Merge the unemployment data into the Lending Club data
combined_data = lending_club_data.merge(right=unemployment_data, left_on='orig_month', right_index=True)

# Merge the S&P 500 data into the combined dataset
combined_data = combined_data.merge(right=modified_sp, left_on='orig_month', right_on='date')

# Merge the BK data into the combined dataset
combined_data = combined_data.merge(right=bk_data_subset, left_on='orig_month', right_index=True)

In [ ]:
# Inspect the data to make certain that all works as intended
print(combined_data.info(verbose=True))
print(combined_data.head())